<a href="https://colab.research.google.com/github/yl4579/StyleTTS2/blob/main/Colab/StyleTTS2_Demo_LJSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages and download models

In [ ]:
%%shell
git clone https://github.com/yl4579/StyleTTS2.git
cd StyleTTS2
pip install SoundFile torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
sudo apt-get install espeak-ng
git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LJSpeech
mv StyleTTS2-LJSpeech/Models .

### Load models

In [ ]:
import os

if not os.path.isdir("Modules"):
    %cd ../

!pwd
lang_id = 'ar'
checkpoint_dir = "Checkpoint_ar_new_aux_whisper_large"
#checkpoint_dir = "Models/LJSpeech_cs_wavlm"
#checkpoint_dir = "Models/LJSpeech_ar_en_whisper"
config_file_name = "config.yml"

import IPython.display as ipd
import torch
import noisereduce as nr
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

import nltk
nltk.download('punkt')

# load packages
import time
import random
import re
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(ref_dicts):
    reference_embeddings = {}
    for key, path in ref_dicts.items():
        wave, sr = librosa.load(path, sr=24000)
        audio, index = librosa.effects.trim(wave, top_db=30)
        if sr != 24000:
            audio = librosa.resample(audio, sr, 24000)
        mel_tensor = preprocess(audio).to(device)

        with torch.no_grad():
            ref = model.style_encoder(mel_tensor.unsqueeze(1))
        reference_embeddings[key] = (ref.squeeze(1), audio)

    return reference_embeddings

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language=lang_id, preserve_punctuation=True, with_stress=True, words_mismatch='ignore')

#config = yaml.safe_load(open( checkpoint_dir + "/config.yml" ))
config = yaml.safe_load(open( checkpoint_dir + "/" + config_file_name ))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model = build_model(recursive_munch(config['model_params']), text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

files = [f for f in os.listdir( checkpoint_dir + "/") if f.startswith('epoch_2nd') and f.endswith('.pth')]
sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

model_path = checkpoint_dir + "/" + sorted_files[-1]
print("Loading model", model_path)
params_whole = torch.load(model_path, map_location='cpu')
params = params_whole['net']

for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

def inference(text, noise, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    text = text.replace('"', '')
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)

    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
        text_mask = length_to_mask(input_lengths).to(tokens.device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise,
              embedding=bert_dur[0].unsqueeze(0), num_steps=diffusion_steps,
              embedding_scale=embedding_scale).squeeze(0)

        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)
        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_dur[-1] += 5

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))

    return out.squeeze().cpu().numpy()


def inference_100ms(
        text: str,
        noise,
        *,
        diffusion_steps: int = 5,
        embedding_scale: float = 1.0,
        sample_rate: int = 24000,
        trim_ms: int = 100,
        remove_pause: bool = True,
):
    """Standard StyleTTS2 inference with built-in tail-trim & pause removal.

    Differences from the original reference:
      • *No* extra pause phonemes/punctuation when ``remove_pause`` is *True*.
      • Last ``trim_ms`` ms of audio are discarded to remove trailing artifacts.
      • Keeps the original behaviour of adding +5 frames to the final token’s
        predicted duration.
    """

    # 1) TEXT PRE-PROCESS
    text = text.strip().replace('"', '')
    if remove_pause:
        text = re.sub(r"[.,;:!?]", "", text)

    ps = global_phonemizer.phonemize([text])
    phonemes = word_tokenize(ps[0])
    if remove_pause:
        phonemes = [p for p in phonemes if p.lower() not in ("sp", "sil", "pau")]
    ps = " ".join(phonemes)

    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
        text_mask = length_to_mask(input_lengths).to(tokens.device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(
            noise,
            embedding=bert_dur[0].unsqueeze(0),
            num_steps=diffusion_steps,
            embedding_scale=embedding_scale,
        ).squeeze(0)

        s, ref = s_pred[:, 128:], s_pred[:, :128]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)
        x, _ = model.predictor.lstm(d)
        duration = torch.sigmoid(model.predictor.duration_proj(x)).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        # preserve original tweak: lengthen final token a bit
        pred_dur[-1] += 5

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            span = int(pred_dur[i])
            pred_aln_trg[i, c_frame:c_frame + span] = 1
            c_frame += span

        en = d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device)
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        wav = model.decoder(
            t_en @ pred_aln_trg.unsqueeze(0).to(device),
            F0_pred,
            N_pred,
            ref.squeeze().unsqueeze(0),
        )

    # 5) POST-PROCESS – trim tail
    audio = wav.squeeze().cpu().numpy()
    samples_trim = int(sample_rate * trim_ms / 1000)
    if audio.shape[-1] > samples_trim:
        audio = audio[:-samples_trim]

    return audio


def LFinference(text, s_prev, noise, alpha=0.7, diffusion_steps=5, embedding_scale=1):
  text = text.strip()
  text = text.replace('"', '')
  ps = global_phonemizer.phonemize([text])
  ps = word_tokenize(ps[0])
  ps = ' '.join(ps)

  tokens = textclenaer(ps)
  tokens.insert(0, 0)
  tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

  with torch.no_grad():
      input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
      text_mask = length_to_mask(input_lengths).to(tokens.device)

      t_en = model.text_encoder(tokens, input_lengths, text_mask)
      bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
      d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

      s_pred = sampler(noise,
            embedding=bert_dur[0].unsqueeze(0), num_steps=diffusion_steps,
            embedding_scale=embedding_scale).squeeze(0)

      if s_prev is not None:
          # convex combination of previous and current style
          s_pred = alpha * s_prev + (1 - alpha) * s_pred

      s = s_pred[:, 128:]
      ref = s_pred[:, :128]

      d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

      x, _ = model.predictor.lstm(d)
      duration = model.predictor.duration_proj(x)
      duration = torch.sigmoid(duration).sum(axis=-1)
      pred_dur = torch.round(duration.squeeze()).clamp(min=1)

      pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
      c_frame = 0
      for i in range(pred_aln_trg.size(0)):
          pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
          c_frame += int(pred_dur[i].data)

      # encode prosody
      en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
      F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
      out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                              F0_pred, N_pred, ref.squeeze().unsqueeze(0))

  return out.squeeze().cpu().numpy(), s_pred


def _trim_tail_if_voiceless(audio, sample_rate, trim_ms=100, voice_threshold=0.02):
    samples_trim = int(sample_rate * trim_ms / 1000)
    if samples_trim <= 0 or len(audio) <= samples_trim:
        return audio
    tail = audio[-samples_trim:]
    energy = np.abs(tail)
    # check if any voiced part (above threshold) is in last 100ms
    if np.any(energy > voice_threshold):
        return audio  # do NOT trim if voiced segment exists in tail
    return audio[:-samples_trim]

def _crossfade_concat(wavs, crossfade_ms=50, sample_rate=24000):
    if not wavs:
        return np.array([])
    if len(wavs) == 1:
        return wavs[0]

    crossfade_samples = int(sample_rate * crossfade_ms / 1000)
    result = wavs[0]

    for w in wavs[1:]:
        if len(result) > crossfade_samples and len(w) > crossfade_samples:
            fade_out = np.linspace(1, 0, crossfade_samples)
            fade_in = np.linspace(0, 1, crossfade_samples)
            overlap = result[-crossfade_samples:] * fade_out + w[:crossfade_samples] * fade_in
            result = np.concatenate([result[:-crossfade_samples], overlap, w[crossfade_samples:]])
        else:
            result = np.concatenate([result, w])

    return result
    
def _append_silence(audio, sample_rate, silence_ms=200, energy_threshold=0.003):
    silence_len = int(sample_rate * silence_ms / 1000)
    tail = audio[-silence_len:]
    if np.max(np.abs(tail)) < energy_threshold:
        noise_segment = tail
    else:
        # Extract low-energy segments for noise reference
        noise_segment = _estimate_noise(audio, sample_rate, segment_ms=silence_ms, energy_threshold=energy_threshold)
        if noise_segment is None:
            noise_segment = np.zeros(silence_len, dtype=audio.dtype)

    # Repeat noise segment until desired length is reached
    repeat_count = int(np.ceil(silence_len / len(noise_segment)))
    extended_noise = np.tile(noise_segment, repeat_count)[:silence_len]
    return np.concatenate([audio, extended_noise])

def _estimate_noise(audio, sample_rate, segment_ms=200, energy_threshold=0.02):
    segment_len = int(sample_rate * segment_ms / 1000)
    noise_segments = []
    for start in range(0, len(audio), segment_len):
        end = start + segment_len
        seg = audio[start:end]
        if len(seg) < segment_len:
            break
        if np.max(np.abs(seg)) < energy_threshold:
            noise_segments.append(seg)
    if not noise_segments:
        return None
    return np.concatenate(noise_segments)

def _denoise_audio(audio, noise_reference, reduction=0.6):
    if noise_reference is None or len(noise_reference) == 0:
        return audio
    return nr.reduce_noise(y=audio, y_noise=noise_reference, sr=24000, prop_decrease=reduction)

def LFinference_trim_100ms(
        text: str,
        s_prev=None,
        noise=None,
        *,
        alpha: float = 0.7,
        diffusion_steps: int = 5,
        embedding_scale: float = 1.0,
        sample_rate: int = 24000,
        trim_ms: int = 100,
        remove_pause: bool = True,
        end_silence_ms: int = 200,
        noise_reduction_threshold: float = 0.6,
        noise_estimage_energy_threshold: float = 0.02,
        append_silence_energy_threshold: float = 0.003
):
    text = text.strip().replace('"', '')
    #if remove_pause:
        #text = re.sub(r"[.,;:!?]", "", text)

    ps = global_phonemizer.phonemize([text])
    phonemes = word_tokenize(ps[0])
    if remove_pause:
        phonemes = [p for p in phonemes if p.lower() not in ("sp", "sil", "pau")]
    ps = " ".join(phonemes)

    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
        text_mask = length_to_mask(input_lengths).to(tokens.device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(
            noise,
            embedding=bert_dur[0].unsqueeze(0),
            num_steps=diffusion_steps,
            embedding_scale=embedding_scale,
        ).squeeze(0)

        if s_prev is not None:
            s_pred = alpha * s_prev + (1.0 - alpha) * s_pred

        s, ref = s_pred[:, 128:], s_pred[:, :128]
        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)
        x, _ = model.predictor.lstm(d)
        duration = torch.sigmoid(model.predictor.duration_proj(x)).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c = 0
        for i in range(pred_aln_trg.size(0)):
            span = int(pred_dur[i])
            pred_aln_trg[i, c:c + span] = 1
            c += span

        en = d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device)
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        wav = model.decoder(
            t_en @ pred_aln_trg.unsqueeze(0).to(device),
            F0_pred,
            N_pred,
            ref.squeeze().unsqueeze(0),
        )

    audio = wav.squeeze().cpu().numpy()
    noise_ref = _estimate_noise(audio, sample_rate, energy_threshold=noise_estimage_energy_threshold)
    audio = _denoise_audio(audio, noise_ref, reduction=noise_reduction_threshold)
    audio = _trim_tail_if_voiceless(audio, sample_rate, trim_ms)
    audio = _append_silence(audio, sample_rate, end_silence_ms, energy_threshold=append_silence_energy_threshold)
    return audio, s_pred

### Synthesize speech

In [ ]:
# @title Input Text { display-mode: "form" }
# synthesize a text
#text = "StyleTTS 2 is a text-to-speech model that leverages style diffusion and adversarial training with large speech language models to achieve human-level text-to-speech synthesis." # @param {type:"string"}
#text = "Příběh strýčka Martina je jiná kniha, než na jaké jste zvyklí. A dost možná to ani není kniha pro vás. Proto také vychází v malém nákladu, jen pro úzký okruh lidí. Není to ani historická beletrie, ani fantazy, ani soudobá próza. Nejblíže má k iniciačnímu románu."
#text = "Tři roky psaná kniha, kterou není lehké zařadit. V prvé fázi deníček studentky, která hledá svého strýčka. V druhé fázi vidiny, stíny minulosti. Dechberoucí obrazy bitev, zkázy, rozcestí české a evropské minulosti. To je Příběh strýčka Martina."
#text = "Historická rovina knihy se pohybuje kolem roku 950, v době boje knížete Boleslava s německým králem Otou. Historický příběh vlastně začíná smrtí hlavního hrdiny, šéfšpicla a šedé eminence české země Martina z Wartberka a ztraceným zemským tributem, který měl konvoj dovést k říšskému králi."
#text = "Horečnou snahou zajistit zemi, která se najednou ocitla ve smrtelném ohrožení, protože kdo ví, co všechno se ztratilo s Wartberkovou smrtí."
#text = "Současný příběh je hledání Veroniky po strýčkovi, který se ztratil. Kam a jak, to vlastně nikdo neví. Až teď začíná Veronika nacházet spojnice mezi svými vizemi a sny se současností. A uvědomuje si, že Martin historický je tím Martinem současným, že najít jednoho znamená najít druhého. Kéž by to bylo tak jednoduché."
#text = "Jenže, darmo se neříká, že cesta je cíl. Platí to i v tomto případě - projít cestu je podstatné."
#text = "V knize se potkáte se dvěmi historickými bitvami, které dnes už vlastně upadly v zapomnění."
#text = "Bitvou u Lechu v roce 955, kde padl prakticky celý český vojenský sbor v boji proti Maďarům - zdarma se dozvíte, z čeho je jméno ""maďaři"" odvozeno - a s bitvou u Nového hradu v roce 950, kde - a o tom kroniky cudně mlčí - porazil kníže Boleslav krále Otu a učinil z Čech plnoprávného souseda a souputníka Říše."
#text = "Setkáte se tu s řadou historických postav. Vlastně všechny postavy v knize jsou postavy žijící, historické, které mají nějaký svůj předobraz. Výjimkou je Maxmilián ze Schweringenu, komoří Oty. Důvod? Toho skutečného nemám rád a tak jsem ho nechtěl ani zmiňovat. Snad mi prominete."
#text = "إذا كان هناك يوم عمل واحد فقط بين عطلتين رسميتين، يعتبر هذا اليوم جزءا من العطلة"
#text = "اللي ما يعرف الصقر يشويه و من طول الغيبات جاب الغنايم"
#text = '"Umíš to přeložit?" Zeptal se mě."'
#text = '"Petra je latinsky skála, ale zároveň je to jméno - Petr. Je odvozené ze slova skála, že?" Zeptal se Martin.'
#text = 'A pak mi to došlo. Takže se nedá říct, jestli Ježíš mluví o Petrovi nebo o skále?'
#text = 'اكتشف بعد فتره طويله مافي شي في الحياة غلط او كذب او ليس صحيح لأ هناك وجهات نظر مختلفه انت تؤمن بهذا الشي'
#text = 'وتطرق الجانبان إلى تطورات الملف السوري، وأكدا أهمية التزام المجتمع الدولي بالمعاهدات والاتفاقيات الدولية ذات الصلة، كما شددا على دور المنظمة في ضمان التنفيذ الفعّال لاتفاقية حظر استخدام الأسلحة الكيميائية.'
#text = 'وأشاد سعادة المدير العام لمنظمة حظر الأسلحة الكيميائية، خلال الاجتماع، بدور دولة قطر في تمثيل مصالح الجمهورية العربية السورية في المنظمة،  ودعمها للجهود الدولية الرامية إلى تحقيق الأمن والاستقرار على المستويين الإقليمي والدولي، والتزامها بمبادئ القانون الدولي'
text = 'أَلِف با تا ثا جِيم حَا خَا دَال ذَال را زَاي سِين شِين صَاد ضَاد طَا ظَا عَين غَين فَا قَاف كَاف لَام مِيم نُون هَا وَاو يَا'

#### Basic synthesis (5 diffusion steps)

In [ ]:
#torch.manual_seed(17484051992422920962)
#print("seed", torch.initial_seed())
print("seed", torch.seed())
start = time.time()
noise = torch.randn(1,1,256).to(device)
wav = inference(text, noise, diffusion_steps=10, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
display(ipd.Audio(wav, rate=24000))

#### With higher diffusion steps (more diverse)
Since the sampler is ancestral, the higher the stpes, the more diverse the samples are, with the cost of slower synthesis speed.

In [ ]:
start = time.time()
noise = torch.randn(1,1,256).to(device)
wav = inference(text, noise, diffusion_steps=10, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
import IPython.display as ipd
display(ipd.Audio(wav, rate=24000))

### Speech expressiveness
The following section recreates the samples shown in [Section 6](https://styletts2.github.io/#emo) of the demo page.

#### With embedding_scale=1
This is the classifier-free guidance scale. The higher the scale, the more conditional the style is to the input text and hence more emotional.

In [ ]:
texts = {}
texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

for k,v in texts.items():
    noise = torch.randn(1,1,256).to(device)
    wav = inference(v, noise, diffusion_steps=10, embedding_scale=1)
    print(k + ": ")
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### With embedding_scale=2

In [ ]:
texts = {}
texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

for k,v in texts.items():
    noise = torch.randn(1,1,256).to(device)
    wav = inference(v, noise, diffusion_steps=10, embedding_scale=2) # embedding_scale=2 for more pronounced emotion
    print(k + ": ")
    display(ipd.Audio(wav, rate=24000, normalize=False))

### Long-form generation
This section includes basic implementation of Algorithm 1 in the paper for consistent longform audio generation. The example passage is taken from [Section 5](https://styletts2.github.io/#long) of the demo page.

In [ ]:
passage = '''
Příběh strýčka Martina je jiná kniha, než na jaké jste zvyklí.
A dost možná to ani není kniha pro vás.
Proto také vychází v malém nákladu, jen pro úzký okruh lidí.
Není to ani historická beletrie, ani fantazy, ani soudobá próza.
Nejblíže má k iniciačnímu románu.
Kniha střídá neumělé vyprávění studenty a barvité sny, vize.
Budete si stěžovat, že nesnášíte její současné části - nebo, že nemáte rádi ty historické.
Bude vám chybět konec nebo přebývat jeho mnohoznačnost.
Pak to zřejmě není kniha pro vás nebo pro tuto vaši životní etapu.
Možná se začtete a až odtrhnete unavené oči o mnoho hodin později, budete svět vidět jinak, s pochybnostmi.
Tři roky psaná kniha, kterou není lehké zařadit.
V prvé fázi deníček studentky, která hledá svého strýčka.
V druhé fázi vidiny, stíny minulosti.
Dechberoucí obrazy bitev, zkázy, rozcestí české a evropské minulosti.
To je Příběh strýčka Martina.
Historická rovina knihy se pohybuje kolem roku 950, v době boje knížete Boleslava s německým králem Otou.
Historický příběh vlastně začíná smrtí hlavního hrdiny, šéfšpicla a šedé eminence české země Martina z Wartberka a ztraceným zemským tributem, který měl konvoj dovést k říšskému králi.
Horečnou snahou zajistit zemi, která se najednou ocitla ve smrtelném ohrožení, protože kdo ví, co všechno se ztratilo s Wartberkovou smrtí.
Současný příběh je hledání Veroniky po strýčkovi, který se ztratil.
Kam a jak, to vlastně nikdo neví.
Až teď začíná Veronika nacházet spojnice mezi svými vizemi a sny se současností.
A uvědomuje si, že Martin historický je tím Martinem současným, že najít jednoho znamená najít druhého.
Kéž by to bylo tak jednoduché...
Jenže, darmo se neříká, že cesta je cíl.
Platí to i v tomto případě - projít cestu je podstatné.
V knize se potkáte se dvěmi historickými bitvami, které dnes už vlastně upadly v zapomnění.
Bitvou u Lechu v roce 955, kde padl prakticky celý český vojenský sbor v boji proti Maďarům. Zdarma se dozvíte, z čeho je jméno "maďaři" odvozeno.
A s bitvou u Nového hradu v roce 950, kde, a o tom kroniky cudně mlčí, porazil kníže Boleslav krále Otu a učinil z Čech plnoprávného souseda a souputníka Říše.
Setkáte se tu s řadou historických postav.
Vlastně všechny postavy v knize jsou postavy žijící, historické, které mají nějaký svůj předobraz.
Výjimkou je Maxmilián ze Schweringenu, komoří Oty.
Důvod?
Toho skutečného nemám rád a tak jsem ho nechtěl ani zmiňovat. Snad mi prominete...
''' # @param {type:"string"}

# passage = '''
# Tu fotku jsem našla ve výpravné fotografické publikaci o Pražském hradu a hned, jak jsem ji viděla, jsem věděla, že tohle je to správné místo, skála.
# Protože abych pravdu řekla, neočekávala jsem, že Martin by měl v oblibě skálu jakožto přírodní útvar.
# V jeho přírodě je skála symbolem, náznakem.
# A skála v podobě přesmyčky latinského petra je ideálním symbolem. Jsem si skoro jistá, že tím místem schůzky je skála v podobě svatého Petra.
# Fotografie pochází z vnější zdi kaple svatého Kříže na druhém nádvoří Pražského hradu a pamatuju si, jak jsme kdysi, to mi bylo něco málo přes deset, kolem té sochy šli.
# Martin mi ukázal ten nápis a přečetl ho: "Tu es Petrus et super hanc petram aedificabo ecclesiam meam."
# "Umíš to přeložit?" Zeptal se mě.
# Šárka s námi nebyla. Ležela doma nemocná a vztekala se, že ji mamka nepustila.
# Začala jsem slabikovat: "Ty, jsi, Petr" a pak už jsem si na ten verš z Bible vzpomněla a dořekla ho: "Petr, Skála - a nad tou skálou vybuduji svoji církev."
# "Pamatuješ si to dobře, Vítězko, ale to tam není."
# Přelétla jsem očima ten nápis, slovo po slově a s drzostí desetiletého vševěda řekla: "Ale je!"
# "Petra je latinsky skála, ale zároveň je to jméno - Petr. Je odvozené ze slova skála, že?" Zeptal se Martin.
# "Ano, to je," zamyslela jsem se.
# A pak mi to došlo: "Takže se nedá říct, jestli Ježíš mluví o Petrovi nebo o skále?"
# "To ne. Z latinského překladu se to opravdu říct nedá. Proto to do češtiny překládají různě, ne každý překladatel ten dvojsmysl akceptoval nebo znal originál."
# "Jak ten verš překládá kralická?"
# Zavrtěla jsem hlavou, že nevím. Nemám kralickou ráda. Radši mám ekumenický překlad, který je čtivější. Obzvlášť pro desetileté dítě.
# '''
passage = '''
في التقرير الأخير عن تطورات المشروع، قيل إن الفريق سيقوم بـ تطرق شامل لكل الجوانب، لكن عند النطق ظهرت كلمة تطرق وكأنها تبدأ بحرف الطاء بدل التاء، وظهرت كلمة تطورات وكأنها تبدأ بحرف الطاء بدلًا من التاء. كذلك تكرر الأمر عند الحديث عن تقييم النتائج، حيث نطقها النظام تكيم بدلًا من تقييم، مما جعل حرف القاف يتحول إلى كاف، وأيضًا في عبارة "هذه قوة إضافية"، تحولت إلى "كوة إضافية". أما عند ذكر الصلة بين الأقسام، فقد قلب النظام الحرف فقرأها السلة بدلًا من الصلة، والعكس حصل في كلمة الأسلحة حيث ظهرت "الأصلحة" بدلًا من "الأسلحة". ومن المشكلات المتكررة أن كلمة منظمة لا تُقرأ كما ينبغي، إذ ينطقها TTS إما "منظمه" أو "منظمات"، فيختفي الفرق بين المفرد والجمع. وأخيرًا في الخطة التعليمية وردت عبارة "هذا البرنامج مقسم إلى المستويين الأول والثاني"، لكن المحرك لم ينطق كلمة المستويين بشكل واضح، بل جعلها أحيانًا "المستويان" وأحيانًا بشكل غير مفهوم. وعند تكرار هذه الكلمات مرة أخرى، نجد أن تطرق، تطورات، تقييم، قوة، الصلة، الأسلحة، منظمة، و المستويين جميعها تظهر مشكلات متكررة في النطق تجعل السامع يدرك أن النظام لا يميز بين الطاء والتاء، ولا بين القاف والكاف، ولا بين السين والصاد، كما يخطئ في التاء المربوطة ويعجز عن نطق المستويين بوضوح.
'''

In [ ]:
#torch.manual_seed(2)
#torch.manual_seed(9)
#torch.manual_seed(12773624985526461855)
torch.manual_seed(17779876142042974804)
print("seed", torch.initial_seed())
#print("seed", torch.seed())
sentences = passage.split('.') # simple split by comma
wavs = []
s_prev = None
for text in sentences:
    if text.strip() == "": continue
    text += '. «' # add it back
    text = text.replace("(", ", ").replace(")", ", ")
    noise = torch.randn(1,1,256).to(device)
    #wav = inference_100ms(text, noise, diffusion_steps=5, embedding_scale=1.2)
    wav, s_prev = LFinference(text, s_prev, noise, alpha=0.7, diffusion_steps=10, embedding_scale=1.5)
    #wav, s_prev = LFinference_trim_100ms(text, s_prev, noise, alpha=0.7, diffusion_steps=10, embedding_scale=1.5)
    #wav, s_prev = LFinference_trim_100ms(text, s_prev, noise, alpha=0.4, diffusion_steps=10, embedding_scale=1.2, trim_ms=350, end_silence_ms=1000, noise_reduction_threshold=0.5, noise_estimage_energy_threshold=0.015, append_silence_energy_threshold=0.0015)
    wavs.append(wav)
display(ipd.Audio(np.concatenate(wavs), rate=24000, normalize=True))
#display(ipd.Audio(_crossfade_concat(wavs, crossfade_ms=1100, sample_rate=24000), rate=24000, normalize=True))